In [1]:
import os
import pandas as pd
import numpy as np 
import datetime as dt
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from api_credentials import client_id, client_secret, redirect_uri
from playlist_membership_check import get_tracks_positions_in_playlists
import json

# Load playlists from JSON file
with open('playlists.json', 'r') as file:
    playlists_dict = json.load(file)


# Initialize the Spotify client with client credentials for public data access
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

Read in the AU Daily 200 Spotify Chart  
Manually downloaded CSV from https://charts.spotify.com/charts/view/regional-au-daily/latest

In [2]:
# Load the CSV file
file_path = 'raw_data/regional-au-daily-2024-02-11.csv'
df = pd.read_csv(file_path)

# Extract the date from the file name
file_name = os.path.basename(file_path)  
date_str = file_name.split('-')[-3:]  # Splits the file name and takes the last 3 elements ['2024', '02', '05.csv']
date_str = '-'.join(date_str)  # Joins them back into '2024-02-05.csv'
date = date_str.split('.')[0]  # Removes the file extension, resulting in '2024-02-05'

# Convert the date string to a datetime object
date_datetime = pd.to_datetime(date)

# Format the datetime object to a string in 'DD-MM-YYYY' format
formatted_date = date_datetime.strftime('%d-%m-%Y')

# Add the formatted date as a new column to the DataFrame
df['date'] = formatted_date

Extract each tracks unique id for API call 

In [3]:
track_ids = df['uri'].str[14:]
df['track_id'] = df['uri'].str[14:]
df.drop(columns='uri', inplace=True)

Call Spotify API to retrieve `popularity_score` for each of the top 200 tracks

In [4]:
# Convert the Series to a list for processing
track_ids_list = track_ids.tolist()

# Spotify's `tracks` endpoint can take multiple track IDs at once,
# but to avoid potential rate limits or payload size issues,
# it's a good idea to process them in batches.
# Here, we define a batch size (e.g., 50 tracks per request)
batch_size = 50

# Initialize lists to store the retrieved data
artists = []
track_names = []
popularity_scores = []

for i in range(0, len(track_ids_list), batch_size):
    batch = track_ids_list[i:i+batch_size]
    tracks_info = sp.tracks(batch)
    for track in tracks_info['tracks']:
        # Some tracks might not be available, leading to `None` entries
        if track:
            # Extracting the primary artist name
            artist_name = track['artists'][0]['name'] if track['artists'] else 'N/A'
            artists.append(artist_name)
            
            # Extracting the track name
            track_names.append(track['name'])
            
            # Extracting the popularity score
            popularity_scores.append(track['popularity'])

# Now that we have all the data, put it into a DataFrame
popularity_df = pd.DataFrame({
    'artist_name': artists,
    'track_name': track_names,
    'popularity_score': popularity_scores
})

# Show the DataFrame 
popularity_df.head()


,artist_name,track_name,popularity_score
0,Noah Kahan,Stick Season,98
1,Benson Boone,Beautiful Things,96
2,Jack Harlow,Lovin On Me,99
3,Taylor Swift,Cruel Summer,99
4,Kanye West,CARNIVAL,82


In [5]:
df['popularity_score'] = popularity_df['popularity_score']

In [6]:
# Reorder columns
df = df[['date', 'rank', 'artist_names', 'track_name', 'track_id', 'source', 'peak_rank', 'previous_rank', 'days_on_chart', 'streams', 'popularity_score']]

# Rename columns
df.rename(columns={
    'source': 'label',
    'streams': 'daily_streams',
}, inplace=True)

In [7]:
df.head()

,date,rank,artist_names,track_name,track_id,label,peak_rank,previous_rank,days_on_chart,daily_streams,popularity_score
0,11-02-2024,1,Noah Kahan,Stick Season,0mflMxspEfB0VbI1kyLiAv,Mercury Records/Republic Records,1,1,129,322278,98
1,11-02-2024,2,Benson Boone,Beautiful Things,6tNQ70jh4OwmPGpYy6R2o9,"Night Street Records, Inc./Warner Records Inc.",2,4,24,277842,96
2,11-02-2024,3,Jack Harlow,Lovin On Me,4xhsWYTOGcal8zt0J161CU,Generation Now/Atlantic,1,2,94,260575,99
3,11-02-2024,4,Taylor Swift,Cruel Summer,1BxfuPKGuaTgP7aM0Bbdwr,Taylor Swift,1,3,372,260230,99
4,11-02-2024,5,"Kanye West, Ty Dolla $ign",CARNIVAL,4m4BHvDfzIgj5LbQLuFiIp,YZY,5,-1,1,240450,82


Retrieve additional data from Kworb.net
This will give us additional features/columns like:  
 - `count_at_peak`
 - `rank_movement`
 - `streams+`
 - `7day_streams`
 - `7day_streams_movement` 
 - `total_streams`

In [8]:
# Read the HTML table into a DataFrame
kworb_df = pd.read_html('https://kworb.net/spotify/country/au_daily.html')[0]

# Get the current date minus 2 days to account for data lag
current_date_minus_2 = dt.datetime.now().date() - dt.timedelta(days=2)

# Add the current date as a new column to the DataFrame
kworb_df['DATE'] = current_date_minus_2

In [9]:
kworb_df = kworb_df.rename(columns={
    'Pos' : 'rank',
    'P+': 'rank_change',
    'Artist and Title' : 'artist_title',
    'Days' : 'days_on_chart',
    'Pk' : 'peak_rank',
    '(x?)' : 'days_at_peak',
    'Streams' : 'daily_streams',
    'Streams+' : 'daily_streams_change',
    '7Day' : '7Day_streams',
    '7Day+' : '7Day_streams_change',
    'Total' : 'total_streams',
})

Need to change "=" to "0", remove "+", and handle "RE"

In [10]:
# Remove "+" and ensure "=" is replaced with "0"
kworb_df['rank_change'] = kworb_df['rank_change'].str.replace('+', '').str.replace('=', '0', regex=False)

# Attempt to convert the column to numeric without coercing errors to identify potential non-numeric values
try:
    kworb_df['rank_change'] = pd.to_numeric(kworb_df['rank_change'])
    print("Conversion successful. No NaN values detected.")
except ValueError as e:
    print("Conversion failed due to non-numeric values:", e)

# Optionally, inspect the column for any remaining non-numeric values or anomalies
# This step is just for verification and won't change the DataFrame
non_numeric = kworb_df[pd.to_numeric(kworb_df['rank_change'], errors='coerce').isna()]
if not non_numeric.empty:
    print("Non-numeric values found:\n", non_numeric)
else:
    print("No non-numeric values found.")

Conversion failed due to non-numeric values: Unable to parse string "NEW" at position 4
Non-numeric values found:
      rank rank_change                                      artist_title  \
4       5         NEW          Kanye West - CARNIVAL (w/ Ty Dolla $ign)   
7       8         NEW        Kanye West - BACK TO ME (w/ Ty Dolla $ign)   
8       9         NEW              Kanye West - BURN (w/ Ty Dolla $ign)   
9      10         NEW          Kanye West - FUK SUMN (w/ Ty Dolla $ign)   
14     15         NEW              Kanye West - PAID (w/ Ty Dolla $ign)   
15     16         NEW   Kanye West - KEYS TO MY LIFE (w/ Ty Dolla $ign)   
16     17         NEW             Kanye West - DO IT (w/ Ty Dolla $ign)   
21     22         NEW           Kanye West - HOODRAT (w/ Ty Dolla $ign)   
22     23         NEW         Kanye West - PAPERWORK (w/ Ty Dolla $ign)   
28     29         NEW          Kanye West - VULTURES (w/ Ty Dolla $ign)   
33     34         NEW   Kanye West - BEG FORGIVENESS (w/ Ty 

/var/folders/px/x65fp4pn31s5cv4gsbh_3cl80000gn/T/ipykernel_65828/641918597.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  kworb_df['rank_change'] = kworb_df['rank_change'].str.replace('+', '').str.replace('=', '0', regex=False)


In [11]:
# Extract numeric values from 'COUNT_AT_PEAK'
kworb_df['days_at_peak'] = kworb_df['days_at_peak'].str.extract('(\d+)', expand=False)

# Convert to numeric, allowing NaNs to remain
kworb_df['days_at_peak'] = pd.to_numeric(kworb_df['days_at_peak'], errors='coerce')

In [12]:
columns_to_keep = kworb_df[['artist_title', 'rank_change', 'days_at_peak', 'daily_streams_change', '7Day_streams', '7Day_streams_change', 'total_streams']]
columns_to_keep.head(3)

,artist_title,rank_change,days_at_peak,daily_streams_change,7Day_streams,7Day_streams_change,total_streams
0,Noah Kahan - Stick Season,0,16.0,7008.0,2328168,-2826,23006053
1,Benson Boone - Beautiful Things,2,4.0,15313.0,1953188,38942,4691217
2,Jack Harlow - Lovin On Me,-1,72.0,-15895.0,1941280,-10785,29886337


In [13]:
print(f"Spotify Daily 200 columns:", df.columns.tolist())
print(f"Kworb Daily 200 columns:", kworb_df.columns.tolist())

Spotify Daily 200 columns: ['date', 'rank', 'artist_names', 'track_name', 'track_id', 'label', 'peak_rank', 'previous_rank', 'days_on_chart', 'daily_streams', 'popularity_score']
Kworb Daily 200 columns: ['rank', 'rank_change', 'artist_title', 'days_on_chart', 'peak_rank', 'days_at_peak', 'daily_streams', 'daily_streams_change', '7Day_streams', '7Day_streams_change', 'total_streams', 'DATE']


In [14]:
df['rank_change'] = kworb_df['rank_change']
df['days_at_peak'] = kworb_df['days_at_peak']
df['daily_streams_change'] = kworb_df['daily_streams_change']
df['7Day_streams'] = kworb_df['7Day_streams']
df['7Day_streams_change'] = kworb_df['7Day_streams_change']
df['total_streams'] = kworb_df['total_streams']


In [15]:
columns_order =  [
    'date',
    'artist_names',
    'track_name',
    'rank',
    'previous_rank',
    'rank_change',
    'peak_rank',
    'days_at_peak',
    'days_on_chart',
    'popularity_score',
    'daily_streams',
    'daily_streams_change',
    '7Day_streams',
    '7Day_streams_change',
    'total_streams',
    'label',
    'track_id',
]

df = df[columns_order]

In [16]:
df.head()

,date,artist_names,track_name,rank,previous_rank,rank_change,peak_rank,days_at_peak,days_on_chart,popularity_score,daily_streams,daily_streams_change,7Day_streams,7Day_streams_change,total_streams,label,track_id
0,11-02-2024,Noah Kahan,Stick Season,1,1,0,1,16.0,129,98,322278,7008.0,2328168,-2826,23006053,Mercury Records/Republic Records,0mflMxspEfB0VbI1kyLiAv
1,11-02-2024,Benson Boone,Beautiful Things,2,4,2,2,4.0,24,96,277842,15313.0,1953188,38942,4691217,"Night Street Records, Inc./Warner Records Inc.",6tNQ70jh4OwmPGpYy6R2o9
2,11-02-2024,Jack Harlow,Lovin On Me,3,2,-1,1,72.0,94,99,260575,-15895.0,1941280,-10785,29886337,Generation Now/Atlantic,4xhsWYTOGcal8zt0J161CU
3,11-02-2024,Taylor Swift,Cruel Summer,4,3,-1,1,1.0,372,99,260230,-4744.0,1693869,36624,52898316,Taylor Swift,1BxfuPKGuaTgP7aM0Bbdwr
4,11-02-2024,"Kanye West, Ty Dolla $ign",CARNIVAL,5,-1,NEW,5,NaN,1,82,240450,NaN,240450,240450,240450,YZY,4m4BHvDfzIgj5LbQLuFiIp


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   date                  200 non-null    object 
 1   artist_names          200 non-null    object 
 2   track_name            200 non-null    object 
 3   rank                  200 non-null    int64  
 4   previous_rank         200 non-null    int64  
 5   rank_change           200 non-null    object 
 6   peak_rank             200 non-null    int64  
 7   days_at_peak          44 non-null     float64
 8   days_on_chart         200 non-null    int64  
 9   popularity_score      200 non-null    int64  
 10  daily_streams         200 non-null    int64  
 11  daily_streams_change  177 non-null    float64
 12  7Day_streams          200 non-null    int64  
 13  7Day_streams_change   200 non-null    int64  
 14  total_streams         200 non-null    int64  
 15  label                 2

In [18]:
yesterday_df = pd.read_csv('combined_data/combined.csv')
yesterday_df.tail()

,date,artist_names,track_name,rank,previous_rank,rank_change,peak_rank,days_at_peak,days_on_chart,popularity_score,daily_streams,daily_streams_change,7Day_streams,7Day_streams_change,total_streams,label,track_id
1195,10-02-2024,TOTO,Africa,196,-1,RE,52,NaN,1816,87,46921,NaN,46921,-1802,67019067,Columbia,2374M0fQpWi3dLnB54qaLX
1196,10-02-2024,"Drake, Yeat",IDGAF (feat. Yeat),197,-1,RE,5,NaN,120,92,46871,NaN,285179,46871,8726337,OVO/Republic,2YSzYUF3jWqb9YP9VXmpjE
1197,10-02-2024,"Timbaland, Keri Hilson, D.O.E.",The Way I Are,198,169,-29,65,NaN,401,87,46792,-4535.0,349735,-9198,17368126,Interscope/Blackground Records,2bLqfJjuC5syrsgDsZfGmn
1198,10-02-2024,"Metro Boomin, Future, Don Toliver",Too Many Nights (feat. Don Toliver & with Future),199,193,-6,32,NaN,302,90,46741,-1854.0,342170,46741,16057901,Republic Records,2Hh3ETdQKrmSI3QS0hme7g
1199,10-02-2024,Olivia Rodrigo,good 4 u,200,189,-11,1,54.0,848,87,46647,-2097.0,144264,46647,87982982,Olivia Rodrigo PS,4ZtFanR9U6ndgddUvNcjcG


In [19]:
combined_df = pd.concat([yesterday_df, df], ignore_index=True)

In [20]:
print(len(combined_df))

1400


In [21]:
combined_df.to_csv('combined_data/combined.csv', index=False)

## Example usage of `get_tracks_position_in_playlist`

Example: Checking which playlists and positions Noah Kahan's 'Stick Season' is in

In [22]:
# Example: Checking playlists and position for Noah Kahan 'Stick Season'
# using it's track_id
track_ids = [
    '0mflMxspEfB0VbI1kyLiAv', 
    # ... add other track IDs
]

track_positions = get_tracks_positions_in_playlists(client_id, client_secret, redirect_uri, playlists_dict, track_ids)

for track_id, playlists in track_positions.items():
    print(f"Track ID {track_id} is in the following playlists at these positions:")
    for playlist_name, positions in playlists.items():
        positions_str = ", ".join(str(position + 1) for position in positions)  # Create a string of positions, accounting for 0 indexing
        print(f"Playlist: {playlist_name}: Position #{positions_str}")
    if not playlists:
        print("This track is not in any of the given playlists.")

Track ID 0mflMxspEfB0VbI1kyLiAv is in the following playlists at these positions:
Playlist: Hot Hits Australia: Position #5
Playlist: Today's Top Hits: Position #12
